In [6]:
# Imports
from pathlib import Path
from massimal import georeferencing
from tqdm import tqdm
import subprocess

In [39]:
# Paths
input_geotiff_dir = Path('/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2d_R_rs_RGB_GeoTiff')
output_geotiff_dir = Path('/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2d_R_rs_RGB_GeoTiff/Reprojected3')
vrt_dir = Path('/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2d_R_rs_RGB_GeoTiff/VirtualRaster')
vrt_path = vrt_dir / 'TestVRT.tiff'

In [5]:
input_raster_paths = sorted(input_geotiff_dir.glob('*.tiff'))
print(f'{len(input_raster_paths)=}')

len(input_raster_paths)=36


In [35]:
def reproject_rotated_raster_to_nonrotated(input_raster_path,output_raster_path):
    """ Convert raster with rotated geotrasform into non-rotated raster with same crs 
    
    # Explanation of arguments used:
    -overwrite:
        Overwrite existing files without error / warning
    -q:
        Suppress GDAL output (quiet)
    -r near:
        Resampling method: Nearest neighbor
    -of GTiff:
        Output format: GeoTiff  
    """
    gdal_args = ['gdalwarp', '-overwrite', '-q', '-r', 'near', '-of', 'GTiff', 
                 str(input_raster_path), str(output_raster_path)]
    subprocess.run(gdal_args)


In [37]:
for input_raster_path in tqdm(input_raster_paths):
    output_raster_path = output_geotiff_dir / input_raster_path.name
    reproject_rotated_raster_to_nonrotated(input_raster_path,output_raster_path)

100%|██████████| 36/36 [00:16<00:00,  2.21it/s]


In [38]:
rasters_to_merge = sorted(output_geotiff_dir.glob('*.tiff'))
print(f'{len(rasters_to_merge)=}')

len(rasters_to_merge)=36


In [51]:
def build_virtual_raster(input_raster_paths,virtual_raster_path):
    """ Merge multiple images into VRT 
    
    # Explanation of arguments used:
    -q:
        Suppress GDAL output ("quiet")
    -overwrite():
        Overwrite existing files without error / warning
    
    """
    gdal_cmd = (['gdalbuildvrt', '-q', '-overwrite'] + 
            [str(vrt_path)] + 
            [str(path) for path in rasters_to_merge])
    subprocess.run(gdal_cmd)

In [52]:
build_virtual_raster(rasters_to_merge,vrt_path)

In [50]:
# gdal_cmd = (['gdalbuildvrt', '-q', '-overwrite', '-resolution', 'average'] + 
#             [str(vrt_path)] + 
#             [str(path) for path in rasters_to_merge])
# print(gdal_cmd)
# subprocess.run(gdal_cmd)
